
### Warning: High Costs Associated with Using Ragas for [RAGAS](https://docs.ragas.io/en/stable/index.html) Test Dataset Generation
Among the frameworks for RAG evaluation, Ragas is famous, but it is absolutely not recommended to use Ragas for generating a test dataset for evaluation. If you use the Ragas code without modification for a test dataset, you will face **exorbitant charges.**

Therefore, I recommend using the [AutoRAG](https://github.com/Marker-Inc-Korea/AutoRAG) framework developed by MarkrAI.

If you are familiar with Korean, please refer to the following tutorial links:

- [Creating a Test Dataset for Evaluation Using AutoRAG Tutorial](https://github.com/Marker-Inc-Korea/AutoRAG-tutorial-ko)
- [Evaluating RAG Using AutoRAG Tutorial](https://github.com/Marker-Inc-Korea/AutoRAG-tutorial-ko)



In [1]:
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install langchain_openai
!pip install chromadb
!pip install sentence-transformers

In [2]:
from bs4 import BeautifulSoup
import requests

url = "https://www.hankookilbo.com/News/Read/A2024070816360002954"
response = requests.get(url)
html_content = response.content

soup = BeautifulSoup(html_content, "html.parser")
p_tags = soup.find_all("p", class_ ="editor-p")

for p in p_tags:
    print(p.text)

그룹 뉴진스(New Jeans)가 영리한 전략으로 일본 시장을 단숨에 섭렵했다. 이들의 도쿄돔 팬미팅은 입성 자체만으로도 기념비적이었으나, 뉴진스는 단순히 'K팝 최단기 입성'의 의미에 만족하는 대신 이를 새로운 도약의 발판으로 만들었다.
결과는 그야말로 대성공이다. 데뷔 이후 국내에서 신드롬급 인기를 구가함은 물론, 해외 음악 시장에서도 빠르게 주목을 받았던 뉴진스는 이번 도쿄돔 팬미팅을 통해 자신들의 존재감을 확실히 각인시켰다. 물론 이전에도 뉴진스를 향한 일본 음악 팬들의 반응은 뜨거웠으나, 일본 정식 데뷔와 맞물려 개최한 도쿄돔 팬미팅은 일본 시장의 '뉴진스 신드롬'에 제대로 불을 붙이는 계기가 됐다. 팬미팅이 끝난 지 열흘이 훌쩍 지났지만, 여전히 일본 음악계는 뉴진스의 열기로 뜨겁다.
데뷔 이후 뉴진스가 기록한 가파른 성장세와 폭발적인 인기의 근간에 헤겔의 '정반합' 논리에 기반한 영민한 전략이 있었던 것처럼, 이번 팬미팅이 이토록 뜨거운 화제를 모을 수 있었던 배경에도 기존의 흐름을 뒤엎는 새롭고 과감한 시도들이 있었다. 
첫 번째 '반'은 기존 K팝 가수들의 전형적인 팬미팅 구성을 깬 '콘서트 급' 구성이었다. 일반적으로 K팝 가수들의 팬미팅은 콘서트와 달리 무대보단 팬들과 가수들의 소통에 초점을 둔 게임이나 토크 코너 등의 비중이 높게 구성된다. 하지만 뉴진스는 게임 등 부대 코너를 대폭 줄이는 대신 20곡이 넘는 세트리스트를 공연 전반에 채워 넣는 변주를 택했다. '하입 보이' '디토' 'OMG' '슈퍼 샤이' 'ETA' 등 뉴진스의 히트곡 퍼레이드는 물론 일본 유명 아티스트들과의 협업 무대, 현지 정상급 세션들이 참여한 밴드 사운드로 채워진 무대들은 뉴진스와 현지 팬들의 깊은 음악적 소통을 이끌어냄과 동시에 뉴진스만의 음악적 색깔과 지향점을 각인시키는 계기가 됐다.
두 번째 '반'은 관객들의 예상을 깬 뉴진스의 무대였다. 이번 팬미팅에서 뉴진스가 히트곡 메들리나 수록곡 무대를 선보일 것이라는 것은 이미 많은 팬들이 예상했던 바다. 이와 함께

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
import bs4 as bs
from pprint import pprint

url = "https://www.hankookilbo.com/News/Read/A2024070816360002954"

loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs.SoupStrainer(
            "p",
             attrs={"class": "editor-p"},
        )
    )
)
docs = loader.load()

print(len(docs))
print(docs[0])
pprint(docs[0].metadata)

1
page_content='그룹 뉴진스(New Jeans)가 영리한 전략으로 일본 시장을 단숨에 섭렵했다. 이들의 도쿄돔 팬미팅은 입성 자체만으로도 기념비적이었으나, 뉴진스는 단순히 'K팝 최단기 입성'의 의미에 만족하는 대신 이를 새로운 도약의 발판으로 만들었다.결과는 그야말로 대성공이다. 데뷔 이후 국내에서 신드롬급 인기를 구가함은 물론, 해외 음악 시장에서도 빠르게 주목을 받았던 뉴진스는 이번 도쿄돔 팬미팅을 통해 자신들의 존재감을 확실히 각인시켰다. 물론 이전에도 뉴진스를 향한 일본 음악 팬들의 반응은 뜨거웠으나, 일본 정식 데뷔와 맞물려 개최한 도쿄돔 팬미팅은 일본 시장의 '뉴진스 신드롬'에 제대로 불을 붙이는 계기가 됐다. 팬미팅이 끝난 지 열흘이 훌쩍 지났지만, 여전히 일본 음악계는 뉴진스의 열기로 뜨겁다.데뷔 이후 뉴진스가 기록한 가파른 성장세와 폭발적인 인기의 근간에 헤겔의 '정반합' 논리에 기반한 영민한 전략이 있었던 것처럼, 이번 팬미팅이 이토록 뜨거운 화제를 모을 수 있었던 배경에도 기존의 흐름을 뒤엎는 새롭고 과감한 시도들이 있었다. 첫 번째 '반'은 기존 K팝 가수들의 전형적인 팬미팅 구성을 깬 '콘서트 급' 구성이었다. 일반적으로 K팝 가수들의 팬미팅은 콘서트와 달리 무대보단 팬들과 가수들의 소통에 초점을 둔 게임이나 토크 코너 등의 비중이 높게 구성된다. 하지만 뉴진스는 게임 등 부대 코너를 대폭 줄이는 대신 20곡이 넘는 세트리스트를 공연 전반에 채워 넣는 변주를 택했다. '하입 보이' '디토' 'OMG' '슈퍼 샤이' 'ETA' 등 뉴진스의 히트곡 퍼레이드는 물론 일본 유명 아티스트들과의 협업 무대, 현지 정상급 세션들이 참여한 밴드 사운드로 채워진 무대들은 뉴진스와 현지 팬들의 깊은 음악적 소통을 이끌어냄과 동시에 뉴진스만의 음악적 색깔과 지향점을 각인시키는 계기가 됐다.두 번째 '반'은 관객들의 예상을 깬 뉴진스의 무대였다. 이번 팬미팅에서 뉴진스가 히트곡 메들리나 수록곡 무대를 선보일 것이라는 것은 이미 많은 팬들이 예상

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-m3')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-12 06:24:12.338074: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 06:24:12.338140: E external/local_x

In [5]:
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 200,
    chunk_overlap  = 50,
)

split_docs = text_splitter.split_documents(docs)
print(len(split_docs))
print(split_docs[0])

26
page_content='그룹 뉴진스(New Jeans)가 영리한 전략으로 일본 시장을 단숨에 섭렵했다. 이들의 도쿄돔 팬미팅은 입성 자체만으로도 기념비적이었으나, 뉴진스는 단순히 'K팝 최단기 입성'의 의미에 만족하는 대신 이를 새로운 도약의 발판으로 만들었다.결과는 그야말로' metadata={'source': 'https://www.hankookilbo.com/News/Read/A2024070816360002954'}


In [6]:
openai_api_key="sk-proj-ZWsFIRT59B8vUsim3OkUT3BlbkFJbBe0pUgvgDydDnei7iO5"

In [7]:
from langchain_openai import ChatOpenAI

vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings_model)
retriever = vectorstore.as_retriever()

In [8]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(openai_api_key=openai_api_key)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke("기사를 3가지 관점에서 정리해주세요")
print(response)

1. 뉴진스의 현재 성장세와 인기에 대한 평가:
- 뉴진스는 향후 세계 시장을 무대로 한 입지를 넓히기 위해 본격적인 박차를 가하고 있으며, 폭발적인 성장세를 보이고 있다.
- 현지 언론들도 뉴진스의 활약을 높이 평가하고 있으며, 특히 팬미팅에서의 무대는 극찬을 받고 있다.
- 뉴진스는 헤겔의 '정반합' 논리를 기반으로 한 전략을 통해 성장하고 있으며, 새롭고 과감한 시도를 통해 기존의 흐름을 뒤엎는 판도를 만들고 있다.

2. 뉴진스의 강점과 전략:
- 뉴진스의 가장 큰 강점은 새로운 판도를 만들 수 있는 능력이다.
- 뉴진스는 도쿄돔 팬미팅을 통해 자사의 전략의 유효성을 입증하고 있으며, 이를 통해 성공적인 행보를 이루고 있다.

3. 뉴진스의 미래 전망:
- 뉴진스는 세계 시장을 무대로 한 입지를 넓혀가고 있으며, 향후에도 성장세를 이어갈 것으로 전망된다.
- 뉴진스는 새로운 시도와 전략을 통해 계속해서 화제를 모으고 있으며, 미래에도 높은 주목을 받을 것으로 예상된다.


### Gen Test Dataset using AutoRAG


In [9]:
!pip install AutoRAG==0.2.9

In [10]:
!mkdir corpus

In [11]:
import nest_asyncio
nest_asyncio.apply()
from autorag.data.corpus import langchain_documents_to_parquet

corpus_df = langchain_documents_to_parquet(split_docs, 'corpus/corpus.parquet')
corpus_df

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


,doc_id,contents,metadata
0,9bd32393-95d9-4a33-9175-3e4cad5c7363,그룹 뉴진스(New Jeans)가 영리한 전략으로 일본 시장을 단숨에 섭렵했다. 이...,{'source': 'https://www.hankookilbo.com/News/R...
1,30a7353b-e3ae-4ed8-8ec3-f3a1d1f4768f,만족하는 대신 이를 새로운 도약의 발판으로 만들었다.결과는 그야말로 대성공이다. 데...,{'source': 'https://www.hankookilbo.com/News/R...
2,d2c6360a-78be-45b9-930f-0e462bbc5e9e,도쿄돔 팬미팅을 통해 자신들의 존재감을 확실히 각인시켰다. 물론 이전에도 뉴진스를 ...,{'source': 'https://www.hankookilbo.com/News/R...
3,3a75dfc9-9092-49d1-92fc-945a09ab5bd8,일본 시장의 '뉴진스 신드롬'에 제대로 불을 붙이는 계기가 됐다. 팬미팅이 끝난 지...,{'source': 'https://www.hankookilbo.com/News/R...
4,614d3ee6-5c56-4d20-955d-5f4cd6f2839a,기록한 가파른 성장세와 폭발적인 인기의 근간에 헤겔의 '정반합' 논리에 기반한 영민...,{'source': 'https://www.hankookilbo.com/News/R...
5,ca7f22e2-1b1d-46b0-9c67-19b835ddc4d6,흐름을 뒤엎는 새롭고 과감한 시도들이 있었다. 첫 번째 '반'은 기존 K팝 가수들의...,{'source': 'https://www.hankookilbo.com/News/R...
6,4ee292ca-9c82-4be4-87e8-0890b2dc0be4,달리 무대보단 팬들과 가수들의 소통에 초점을 둔 게임이나 토크 코너 등의 비중이 높...,{'source': 'https://www.hankookilbo.com/News/R...
7,7d12ea3c-73fc-41bd-b19a-4f87a841b794,넘는 세트리스트를 공연 전반에 채워 넣는 변주를 택했다. '하입 보이' '디토' '...,{'source': 'https://www.hankookilbo.com/News/R...
8,0ee25322-b00f-41f9-8fdb-4c1424cc52ca,"협업 무대, 현지 정상급 세션들이 참여한 밴드 사운드로 채워진 무대들은 뉴진스와 현...",{'source': 'https://www.hankookilbo.com/News/R...
9,0ff10b22-1a19-421e-bc17-320cd8935e40,각인시키는 계기가 됐다.두 번째 '반'은 관객들의 예상을 깬 뉴진스의 무대였다. 이...,{'source': 'https://www.hankookilbo.com/News/R...


In [12]:
import pandas as pd
from autorag.data.qacreation import generate_qa_llama_index, make_single_content_qa
from llama_index.llms.openai import OpenAI


prompt = """
Generate question and answer pairs for the given passage.

Passage:
{{text}}

Number of questions to generate: {{num_questions}}

Example:
[Q]: 뉴진스 팬미팅에서 하니가 부른 일본 노래는 무엇인가요?
[A]: 마쓰다 세이코의 '푸른 산호초'입니다.

Result:
"""

corpus_df = pd.read_parquet('corpus/corpus.parquet')
llm = OpenAI(model='gpt-3.5-turbo', temperature=1.0, api_key=openai_api_key)
qa_df = make_single_content_qa(corpus_df, content_size=26, qa_creation_func=generate_qa_llama_index, llm=llm,
                               prompt=prompt, question_num_per_content=1, output_filepath='corpus/qa.parquet')

1it [00:10, 10.66s/it]


In [25]:
qa_df.head()

,retrieval_gt,qid,query,generation_gt
0,[[908d31d5-333e-4146-9fee-4b0f827536a5]],e7b0fe4f-b560-4e08-8079-abae9191a4b2,뉴진스의 두 번째 '반'은 무엇을 깰 예정이었나요?,[관객들의 예상을 깨는 것을 계획하고 있었습니다.]
1,[[c1720731-b4c7-4da1-8536-5153562b88c4]],eec2dcd9-0856-4deb-befc-f9279bb560da,'푸른 산호초' 무대를 내세운 뉴진스와 민희진 어도어 대표의 전략은 누구가 선곡해 ...,[민희진 대표가 직접 선곡해 연출했습니다.]
2,[[d61d5bb1-1b6c-43cd-a7fe-e46e87753091]],7281ac50-1e3b-4288-8d0d-e9d63aa2593b,뉴진스가 도쿄돔에서 열린 팬미팅에 섭렵한 나라는 어디인가요?,[일본입니다.]
3,[[ae6bbedd-467d-484a-a8f5-0ba801bc0f87]],887616e9-6b61-4195-8ddc-e9b72a0f3f6d,뉴진스의 가장 큰 강점은 무엇인가요?,[새로운 판도를 만들 수 있다는 것입니다.]
4,[[21e9a16e-edcd-4e32-adea-242f6f3b0f52]],48dd3a60-3e8b-4149-963c-74c39d37836b,다케우치 마리야의 '플라스틱 러브'가 어떤 반응을 일으켰나요?,[팬들의 예상을 훌쩍 뛰어 넘는 파격적인 반응을 일으켰습니다.]


In [13]:
qa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   retrieval_gt   26 non-null     object
 1   qid            26 non-null     object
 2   query          26 non-null     object
 3   generation_gt  26 non-null     object
dtypes: object(4)
memory usage: 960.0+ bytes


In [14]:
print(type(qa_df['retrieval_gt'][0]))
print(type(qa_df['generation_gt'][0]))

<class 'list'>
<class 'list'>


### Evaluation using Ragas
https://docs.ragas.io/en/latest/getstarted/evaluation.html

In [15]:
qa_ds = qa_df[['query', 'generation_gt']]
qa_ds.columns = ['question', 'ground_truth']
qa_ds.head()

,question,ground_truth
0,어떤 무대들이 현지 정상급 세션들과 협업하여 뉴진스와 현지 팬들의 음악적 소통을 이...,[밴드 사운드로 채워진 무대들이 협업을 통해 뉴진스와 현지 팬들의 깊은 음악적 소통...
1,'푸른 산호초' 무대를 내세운 뉴진스와 민희진 어도어 대표는 어떤 전략을 사용했나요?,['푸른 산호초' 무대는 민 대표가 직접 선곡해 연출한 것입니다.]
2,뉴진스는 어떤 전략으로 일본 시장을 섭렵했나요?,[영리한 전략을 사용하여 일본 시장을 섭렵했습니다.]
3,뉴진스의 가장 큰 강점은 무엇인가요?,[새로운 판도를 만들 수 있다는 것입니다.]
4,다케우치 마리야의 '플라스틱 러브'를 부른 멤버는 누구인가요?,[하니입니다.]


In [16]:
qa_ds['ground_truth'] = qa_ds['ground_truth'].apply(lambda x: x[0] )
qa_ds['answer'] = qa_ds['question'].apply(lambda x: rag_chain.invoke(x) )
qa_ds['contexts'] = qa_ds['question'].apply(lambda x: [d.page_content for d in retriever.get_relevant_documents(x)] )
qa_ds.head()

/tmp/ipykernel_3235/2085200181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_ds['ground_truth'] = qa_ds['ground_truth'].apply(lambda x: x[0] )
/tmp/ipykernel_3235/2085200181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_ds['answer'] = qa_ds['question'].apply(lambda x: rag_chain.invoke(x) )
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and 

,question,ground_truth,answer,contexts
0,어떤 무대들이 현지 정상급 세션들과 협업하여 뉴진스와 현지 팬들의 음악적 소통을 이...,밴드 사운드로 채워진 무대들이 협업을 통해 뉴진스와 현지 팬들의 깊은 음악적 소통을...,협업 무대들,"[협업 무대, 현지 정상급 세션들이 참여한 밴드 사운드로 채워진 무대들은 뉴진스와 ..."
1,'푸른 산호초' 무대를 내세운 뉴진스와 민희진 어도어 대표는 어떤 전략을 사용했나요?,'푸른 산호초' 무대는 민 대표가 직접 선곡해 연출한 것입니다.,민 대표가 직접 선곡해 연출한 전략을 사용했다.,[하니의 '푸른 산호초' 무대가 얼마나 큰 의미와 파급력을 지녔는지 가늠된다. 동시...
2,뉴진스는 어떤 전략으로 일본 시장을 섭렵했나요?,영리한 전략을 사용하여 일본 시장을 섭렵했습니다.,뉴진스는 영리한 전략으로 일본 시장을 섭렵했다.,[그룹 뉴진스(New Jeans)가 영리한 전략으로 일본 시장을 단숨에 섭렵했다. ...
3,뉴진스의 가장 큰 강점은 무엇인가요?,새로운 판도를 만들 수 있다는 것입니다.,뉴진스의 가장 큰 강점은 기존의 흐름을 뒤엎고 새로운 판도를 만들 수 있는 전략적인...,[새로운 판도를 만들 수 있다는 것이 뉴진스의 가장 큰 강점이다. 도쿄돔 팬미팅은 ...
4,다케우치 마리야의 '플라스틱 러브'를 부른 멤버는 누구인가요?,하니입니다.,하니 (Hani),"[산호초', 다케우치 마리야의 '플라스틱 러브'는 팬들의 예상을 훌쩍 뛰어 넘는 파..."


In [17]:
from datasets import Dataset
dataset = Dataset.from_pandas(qa_ds)
dataset[0]

{'question': '어떤 무대들이 현지 정상급 세션들과 협업하여 뉴진스와 현지 팬들의 음악적 소통을 이끌어냈나요?',
 'ground_truth': '밴드 사운드로 채워진 무대들이 협업을 통해 뉴진스와 현지 팬들의 깊은 음악적 소통을 이끌어냈습니다.',
 'answer': '협업 무대들',
 'contexts': ["협업 무대, 현지 정상급 세션들이 참여한 밴드 사운드로 채워진 무대들은 뉴진스와 현지 팬들의 깊은 음악적 소통을 이끌어냄과 동시에 뉴진스만의 음악적 색깔과 지향점을 각인시키는 계기가 됐다.두 번째 '반'은 관객들의 예상을 깬",
  '만족하는 대신 이를 새로운 도약의 발판으로 만들었다.결과는 그야말로 대성공이다. 데뷔 이후 국내에서 신드롬급 인기를 구가함은 물론, 해외 음악 시장에서도 빠르게 주목을 받았던 뉴진스는 이번 도쿄돔 팬미팅을 통해 자신들의 존재감을 확실히 각인시켰다. 물론',
  "도쿄돔 팬미팅을 통해 자신들의 존재감을 확실히 각인시켰다. 물론 이전에도 뉴진스를 향한 일본 음악 팬들의 반응은 뜨거웠으나, 일본 정식 데뷔와 맞물려 개최한 도쿄돔 팬미팅은 일본 시장의 '뉴진스 신드롬'에 제대로 불을 붙이는 계기가",
  "넘는 세트리스트를 공연 전반에 채워 넣는 변주를 택했다. '하입 보이' '디토' 'OMG' '슈퍼 샤이' 'ETA' 등 뉴진스의 히트곡 퍼레이드는 물론 일본 유명 아티스트들과의 협업 무대, 현지 정상급 세션들이 참여한 밴드"]}

In [18]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [21]:
from ragas import evaluate
from langchain.embeddings.openai import OpenAIEmbeddings

llm =  ChatOpenAI(openai_api_key=openai_api_key)
embeddings = embeddings_model

result = evaluate(
    dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=llm,
    embeddings=embeddings,
    raise_exceptions=False,
)

result

Evaluating:   0%|          | 0/104 [00:00<?, ?it/s]

{'faithfulness': 0.4978, 'answer_relevancy': 0.5567, 'context_recall': 0.7960, 'context_precision': 0.9637}

In [22]:
eval_result = pd.DataFrame(list(result.items()), columns=['Metric', 'ChatGpt-3.5'])

In [23]:
eval_result

,Metric,ChatGpt-3.5
0,faithfulness,0.497826
1,answer_relevancy,0.556692
2,context_recall,0.796000
3,context_precision,0.963675
